<a href="https://colab.research.google.com/github/afifatanjeemadiba-netizen/CSE423_Project/blob/main/Escape_The_Grid_3D_Adventure_Game_Sumaiya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *
import math
import random
import time

WINDOW_WIDTH = 1000
WINDOW_HEIGHT = 700
MOVE_SPEED = 0.015
CAMERA_TURN_SPEED = 2.5
JUMP_SPEED = 0.18
GRAVITY = -0.015
MAX_FALL_SPEED = -0.5
FRICTION = 0.85
AIR_FRICTION = 0.92
PLAYER_HEIGHT = 1.8
PLAYER_WIDTH = 0.6
PLAYER_HEIGHT_09 = PLAYER_HEIGHT * 0.9
PLAYER_WIDTH_HALF = PLAYER_WIDTH / 2.0

# UTILITY CLASSES

class Vector3:
    def __init__(self, x=0, y=0, z=0):
        self.x, self.y, self.z = x, y, z
    def __add__(self, other):
        return Vector3(self.x + other.x, self.y + other.y, self.z + other.z)
    def __sub__(self, other):
        return Vector3(self.x - other.x, self.y - other.y, self.z - other.z)
    def __mul__(self, scalar):
        return Vector3(self.x * scalar, self.y * scalar, self.z * scalar)
    def length(self):
        return math.sqrt(self.x**2 + self.y**2 + self.z**2)
    def normalize(self):
        length = self.length()
        if length > 0:
            return Vector3(self.x/length, self.y/length, self.z/length)
        return Vector3(0, 0, 0)
    def dot(self, other):
        return self.x * other.x + self.y * other.y + self.z * other.z
class GameObject:
    def __init__(self, position=Vector3(), size=1.0):
        self.position = position
        self.size = size
        self.active = True

        #Get bounding box for collision detection

    def get_bounds(self):
        return {
            'min_x': self.position.x - self.size/2,
            'max_x': self.position.x + self.size/2,
            'min_z': self.position.z - self.size/2,
            'max_z': self.position.z + self.size/2
        }

class PowerUp(GameObject):# Speed Boost, Shield, Rapid Fire
    def __init__(self, position, power_type):
        super().__init__(position, 0.5)
        self.power_type = power_type
        self.rotation = 0
        self.bob_time = 0
        self.pulse_time = 0
    def update(self):
        self.rotation += 1.8
        self.bob_time += 0.1
        self.pulse_time += 0.15
    def draw_torus_manually(self, inner_radius, outer_radius, sides, rings):
        for i in range(rings):
            glBegin(GL_QUADS)
            for j in range(sides):
                for k in range(2):
                    s = (i + k) % rings
                    t = j
                    angle1 = s * 2.0 * math.pi / rings
                    angle2 = t * 2.0 * math.pi / sides
                    cos1, sin1 = math.cos(angle1), math.sin(angle1)
                    cos2, sin2 = math.cos(angle2), math.sin(angle2)
                    r = outer_radius + inner_radius * cos2
                    x = cos1 * r
                    y = sin1 * r
                    z = inner_radius * sin2
                    t_next = (j + 1) % sides
                    angle2_next = t_next * 2.0 * math.pi / sides
                    cos2_next, sin2_next = math.cos(angle2_next), math.sin(angle2_next)
                    r_next = outer_radius + inner_radius * cos2_next
                    x_next = cos1 * r_next
                    y_next = sin1 * r_next
                    z_next = inner_radius * sin2_next
                    if k == 0:
                        glVertex3f(x, y, z)
                        glVertex3f(x_next, y_next, z_next)
                    else:
                        s_next = (i + 1) % rings
                        angle1_next = s_next * 2.0 * math.pi / rings
                        cos1_next, sin1_next = math.cos(angle1_next), math.sin(angle1_next)
                        r_next2 = outer_radius + inner_radius * cos2_next
                        x_next2 = cos1_next * r_next2
                        y_next2 = sin1_next * r_next2
                        z_next2 = inner_radius * sin2_next
                        r2 = outer_radius + inner_radius * cos2
                        x2 = cos1_next * r2
                        y2 = sin1_next * r2
                        z2 = inner_radius * sin2
                        glVertex3f(x_next2, y_next2, z_next2)
                        glVertex3f(x2, y2, z2)
            glEnd()
    def draw(self):# Draw with rotation, bobbing, and pulsing
        if not self.active:
            return
        glPushMatrix()
        bob_height = math.sin(self.bob_time) * 0.2
        pulse = 1.0 + 0.15 * math.sin(self.pulse_time * 3)
        glTranslatef(self.position.x, self.position.y + 0.4 + bob_height, self.position.z)
        glRotatef(self.rotation, 1, 1, 0)
        if self.power_type == "speed":
            glColor3f(0.0, 1.0 * pulse, 0.2)
        elif self.power_type == "shield":
            glColor3f(0.0, 0.6 * pulse, 1.0)
        elif self.power_type == "rapid_fire":
            glColor3f(1.0 * pulse, 0.6, 0.0)
        self.draw_torus_manually(self.size * 0.3, self.size * 0.7, 8, 16)
        if self.power_type == "speed":
            glColor3f(0.0 * pulse, 1.0 * pulse, 0.2 * pulse)
        elif self.power_type == "shield":
            glColor3f(0.0 * pulse, 0.6 * pulse, 1.0 * pulse)
        elif self.power_type == "rapid_fire":
            glColor3f(1.0 * pulse, 0.6 * pulse, 0.0 * pulse)
        self.draw_torus_manually(self.size * 0.4, self.size * 0.8, 8, 16)
        glPopMatrix()


class Arena:
    def __init__(self):
        self.size = 22
        self.tile_states = []
        self.tile_heights = []
        self.init_tiles()
    def init_tiles(self):
        self.tile_states = []
        self.tile_heights = []
        for x in range(-self.size//2, self.size//2):
            row_states = []
            row_heights = []
            for z in range(-self.size//2, self.size//2):
                state = 0
                height = 0
                if abs(x) < 4 and abs(z) < 4:
                    state = 0
                else:
                    rand = random.random()
                    if rand < 0.15:
                        state = 1
                        height = random.uniform(1.5, 3.0)
                    elif rand < 0.20:
                        state = 2
                row_states.append(state)
                row_heights.append(height)
            self.tile_states.append(row_states)
            self.tile_heights.append(row_heights)
    def update(self):# Animate tile heights for elevated tiles
        for x in range(len(self.tile_states)):
            for z in range(len(self.tile_states[0])):
                if self.tile_states[x][z] == 1:
                    base_height = 1.5 + (x + z) % 3 * 0.5
                    self.tile_heights[x][z] = base_height + math.sin(time.time() * 0.8 + x + z) * 0.15
    def get_tile_at(self, world_x, world_z):# Get tile type at world coordinates
        tile_x = int(world_x + self.size//2)
        tile_z = int(world_z + self.size//2)
        if 0 <= tile_x < self.size and 0 <= tile_z < self.size:
            return self.tile_states[tile_x][tile_z]
        return 0
    def get_tile_height(self, world_x, world_z):# Get tile height at world coordinates
        tile_x = int(world_x + self.size//2)
        tile_z = int(world_z + self.size//2)
        if 0 <= tile_x < self.size and 0 <= tile_z < self.size:
            return self.tile_heights[tile_x][tile_z]
        return 0
    def draw(self):# Draw arena tiles and walls
        for x in range(-self.size//2, self.size//2):
            for z in range(-self.size//2, self.size//2):
                tile_x = x + self.size//2
                tile_z = z + self.size//2
                state = self.tile_states[tile_x][tile_z]
                height = self.tile_heights[tile_x][tile_z]
                glPushMatrix()
                glTranslatef(x, height/2, z)
                if state == 2:
                    lava_intensity = 0.9 + 0.3 * math.sin(time.time() * 3 + x + z)
                    glColor3f(1.0, lava_intensity * 0.4, 0.0)
                    glColor3f(1.0 * lava_intensity, 0.6, 0.0)
                    glScalef(1.2, max(height, 0.1) + 0.3, 1.2)
                    glutSolidCube(1)
                    glScalef(1/1.2, 1/(max(height, 0.1) + 0.3), 1/1.2)
                    glColor3f(1.0, lava_intensity * 0.4, 0.0)
                elif state == 1:
                    glColor3f(0.3, 0.3, 0.7)
                else:
                    if (x + z) % 2 == 0:
                        glColor3f(0.8, 0.8, 0.8)
                    else:
                        glColor3f(0.6, 0.6, 0.6)
                glScalef(1, max(height, 0.1), 1)
                glutSolidCube(1)
                glPopMatrix()
        glColor3f(0.15, 0.15, 0.4)
        wall_height = 6
        for i in range(4):
            glPushMatrix()
            if i == 0:
                glTranslatef(0, wall_height/2, -self.size//2 - 0.5)
                glScalef(self.size + 1, wall_height, 1)
            elif i == 1:
                glTranslatef(0, wall_height/2, self.size//2 + 0.5)
                glScalef(self.size + 1, wall_height, 1)
            elif i == 2:
                glTranslatef(-self.size//2 - 0.5, wall_height/2, 0)
                glScalef(1, wall_height, self.size + 1)
            else:
                glTranslatef(self.size//2 + 0.5, wall_height/2, 0)
                glScalef(1, wall_height, self.size + 1)
            glutSolidCube(1)
            glPopMatrix()